In [ ]:
# https://github.com/MorvanZhou/tutorials/blob/master/tensorflowTUT/tf16_classification/full_code.py
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler

In [ ]:
cols = ['huml', 'humw', 'ulnal', 'ulnaw', 'feml', 
        'femw', 'tibl', 'tibw', 'tarl', 'tarw']

bird_data = pd.read_csv("datasets/bird.csv")
bird_data = bird_data.fillna(-1)
bird_type_class = np.unique(bird_data['type'])
bird_summary = bird_data.groupby('type').mean()

In [ ]:
bird_data_no_miss = pd.DataFrame()
for _type in bird_type_class:
    col_data = bird_data[bird_data['type']==_type]
    type_summary = bird_summary[bird_summary.index==_type]
    for col in cols:
        np.place(col_data[col].values, col_data[col] == -1, 
                 np.round(type_summary[col].values, 2))
    bird_data_no_miss = bird_data_no_miss.append(col_data)
bird_data_no_miss = bird_data_no_miss.reset_index(drop=True)

bird_x = bird_data_no_miss[cols].values
bird_y = bird_data_no_miss['type']
encoder = LabelBinarizer()
bird_y = encoder.fit_transform(bird_y)

train_x, test_x, train_y, test_y = train_test_split(bird_x, bird_y, test_size=0.3)

In [ ]:
# mini-max
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(train_x)
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [ ]:
# y = xW + b
# y: (#, 6); x: (#, 10); W: (10, 6); b: (1, 6)
def add_layer(_input, in_size, out_size, activation_fun=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]))
    Wx_plus_b = tf.matmul(_input, Weights) + biases
    
    if activation_fun is None:
        output = Wx_plus_b
    else:
        output = activation_fun(Wx_plus_b)
    return output

def compute_accuracy(test_x, test_y):
    global prediction
    predict = sess.run(prediction, feed_dict={tf_x: test_x})
    correct = tf.equal(tf.argmax(predict, 1), tf.argmax(test_y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    result = sess.run(accuracy, feed_dict={tf_x: test_x, tf_y: test_y})
    return result

In [ ]:
tf_x = tf.placeholder(tf.float32, [None, 10])
tf_y = tf.placeholder(tf.float32, [None, 6])

prediction = add_layer(tf_x, 10, 6, activation_fun=tf.nn.softmax)

# cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(tf_y*tf.log(prediction),
                                     reduction_indices=[1]))
# train = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

In [ ]:
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
with tf.Session() as sess:
    sess.run(init)
    for i in range(2000):
        sess.run(train, {tf_x: train_x, tf_y: train_y})
        if (i+1) % 100 == 0:
            l = sess.run(loss, {tf_x: train_x, tf_y: train_y})
            acc = compute_accuracy(test_x, test_y)
            print('loss:', l, ',accuracy:', acc)